# Collect Demonstration from Keyboard

Collect demonstration data for the given environment.
The task is to pick a mug and place it on the plate. The environment recognizes the success if the mug is on the plate, gthe ripper opened, and the end-effector positioned above the mug.

<img src="./media/teleop_v2.gif" width="480" height="360">

Use WASD for the xy plane, RF for the z-axis, QE for tilt, and ARROWs for the rest of rthe otations. 

SPACEBAR will change your gripper's state, and Z key will reset your environment with discarding the current episode data.

For overlayed images, 
- Top Right: Agent View 
- Bottom Right: Egocentric View
- Top Left: Left Side View
- Bottom Left: Top View

In [1]:
import sys
import random
import numpy as np
import os
from PIL import Image
from mujoco_env.y_env2_remove_block_collect_dataset import SimpleEnvRemoveBlockCollectDataset
from lerobot.datasets.lerobot_dataset import LeRobotDataset

/home/dragon/anaconda3/envs/mujoco_vla/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# If you want to randomize the object positions, set this to None
# If you fix the seed, the object positions will be the same every time
SEED = 0 
# SEED = None <- Uncomment this line to randomize the object positions

REPO_NAME = 'remove_red_block_from_plate_UR5_smolvla_mujoco'
NUM_DEMO = 20 # Number of demonstrations to collect
ROOT = "./demo_data_language" # The root directory to save the demonstrations

# ==================== 随机Block位置配置 ====================
ENABLE_RANDOM_BLOCK = True  # 设置为False禁用随机位置
PLATE_FLAT_RADIUS = 0.102   # 基于精确分析的平整区域半径（米）
Block_size = 0.015
Safe_dis = 0.015

print(f"🎯 配置信息:")
print(f"  随机Block位置: {'启用' if ENABLE_RANDOM_BLOCK else '禁用'}")
print(f"  数据集名称: {REPO_NAME}")
print(f"  Episode数量: {NUM_DEMO}")
print(f"  随机种子: {SEED}")
if ENABLE_RANDOM_BLOCK:
    print(f"  Plate平整区域半径: {PLATE_FLAT_RADIUS:.3f}m")
    print(f"  预计安全生成半径: {PLATE_FLAT_RADIUS- Safe_dis - Block_size * np.sqrt(2):.3f}m")

🎯 配置信息:
  随机Block位置: 启用
  数据集名称: remove_red_block_from_plate_UR5_smolvla_mujoco
  Episode数量: 20
  随机种子: 0
  Plate平整区域半径: 0.102m
  预计安全生成半径: 0.066m


In [3]:
SAMPLING_METHOD = 'uniform'

xml_path = './asset/scene_remove_block.xml'
# Define the environment with random block position support
PnPEnv = SimpleEnvRemoveBlockCollectDataset(
    xml_path, 
    seed=SEED, 
    state_type='joint_angle',
    random_block_position=ENABLE_RANDOM_BLOCK,
    plate_flat_radius=PLATE_FLAT_RADIUS,
    sampling_method=SAMPLING_METHOD
)


-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[31] n_qvel:[28] n_qacc:[28] n_ctrl:[10]
 integrator:[IMPLICITFAST]

n_body:[22]
 [0/22] [world] mass:[0.00]kg
 [1/22] [front_object_table] mass:[1.00]kg
 [2/22] [camera] mass:[0.00]kg
 [3/22] [camera2] mass:[0.00]kg
 [4/22] [camera3] mass:[0.00]kg
 [5/22] [link1] mass:[2.06]kg
 [6/22] [link2] mass:[3.68]kg
 [7/22] [link3] mass:[2.39]kg
 [8/22] [link4] mass:[1.40]kg
 [9/22] [link5] mass:[1.40]kg
 [10/22] [link6] mass:[0.65]kg
 [11/22] [camera_center] mass:[0.00]kg
 [12/22] [tcp_link] mass:[0.32]kg
 [13/22] [rh_p12_rn_r1] mass:[0.07]kg
 [14/22] [rh_p12_rn_r2] mass:[0.02]kg
 [15/22] [rh_p12_rn_l1] mass:[0.07]kg
 [16/22] [rh_p12_rn_l2] mass:[0.02]kg
 [17/22] [body_obj_plate_11] mass:[0.00]kg
 [18/22] [object_plate_11] mass:[0.10]kg
 [19/22] [body_obj_mug_6] mass:[0.00]kg
 [20/22] [object_mug_6] mass:[0.08]kg
 [21/22] [body_obj_block_red] mass:[0.30]kg
body_total_mass:

In [4]:
# ==================== 简单位置测试 ====================
print("🧪 Block位置生成测试")
print("=" * 40)

if ENABLE_RANDOM_BLOCK:
    print(f"📍 采样方法: {SAMPLING_METHOD}")
    # 生成3个测试位置
    for i in range(3):
        PnPEnv.reset()
        block_pose, _, _ = PnPEnv.get_obj_pose()
        print(f"  位置 {i+1}: [{block_pose[0]:.3f}, {block_pose[1]:.3f}, {block_pose[2]:.3f}]")
    print("✅ 随机位置生成正常")
else:
    PnPEnv.reset()
    block_pose, _, _ = PnPEnv.get_obj_pose()
    print(f"📍 固定位置: [{block_pose[0]:.3f}, {block_pose[1]:.3f}, {block_pose[2]:.3f}]")

print("=" * 40)


🧪 Block位置生成测试
📍 采样方法: uniform
随机生成red block位置: [0.240, -0.225, 0.840]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION
  位置 1: [0.240, -0.225, 0.823]
随机生成red block位置: [0.229, -0.263, 0.840]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION
  位置 2: [0.230, -0.263, 0.823]
随机生成red block位置: [0.324, -0.247, 0.840]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION
  位置 3: [0.324, -0.247, 0.823]
✅ 随机位置生成正常


## Define Dataset Fatures and Create your dataset!
The dataset is contained as follows:
```
fps = 20,
features={
    "observation.image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channels"],
    },
    "observation.wrist_image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channel"],
    },
    "observation.state": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["state"], # x, y, z, roll, pitch, yaw
    },
    "action": {
        "dtype": "float32",
        "shape": (7,),
        "names": ["action"], # 6 joint angles and 1 gripper
    },
    "obj_init": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["obj_init"], # just the initial position of the object. Not used in training.
    },
},
```


This will make the dataset on './demo_data' folder, which will look like this,

```
.
├── data
│   ├── chunk-000
│   │   ├── episode_000000.parquet
│   │   └── ...
├── meta
│   ├── episodes.jsonl
│   ├── info.json
│   ├── stats.json
│   └── tasks.jsonl
└── 
```


In [5]:
# ==================== 快速配置选项 ====================

print("🚀 Block位置配置选项")
print("=" * 50)
print()
print("📋 参数说明:")
print("  ENABLE_RANDOM_BLOCK: True=随机位置, False=固定位置")
print("  SAMPLING_METHOD: 'uniform'=均匀随机, 'latin_hypercube'=拉丁超立方")
print("  NUM_DEMO: 要收集的episode数量")
print()
print("🎯 推荐配置:")
print("1. 固定位置: ENABLE_RANDOM_BLOCK = False")
print("2. 均匀随机: ENABLE_RANDOM_BLOCK = True, SAMPLING_METHOD = 'uniform'")
print("3. 拉丁超立方: ENABLE_RANDOM_BLOCK = True, SAMPLING_METHOD = 'latin_hypercube'")
print("=" * 50)


🚀 Block位置配置选项

📋 参数说明:
  ENABLE_RANDOM_BLOCK: True=随机位置, False=固定位置
  SAMPLING_METHOD: 'uniform'=均匀随机, 'latin_hypercube'=拉丁超立方
  NUM_DEMO: 要收集的episode数量

🎯 推荐配置:
1. 固定位置: ENABLE_RANDOM_BLOCK = False
2. 均匀随机: ENABLE_RANDOM_BLOCK = True, SAMPLING_METHOD = 'uniform'
3. 拉丁超立方: ENABLE_RANDOM_BLOCK = True, SAMPLING_METHOD = 'latin_hypercube'


In [6]:
create_new = True
if os.path.exists(ROOT):
    print(f"Directory {ROOT} already exists.")
    ans = input("Do you want to delete it? (y/n) ")
    if ans == 'y':
        import shutil
        shutil.rmtree(ROOT)
    else:
        create_new = False


if create_new:
    dataset = LeRobotDataset.create(
                repo_id=REPO_NAME,
                root = ROOT, 
                robot_type="omy",
                fps=20, # 20 frames per second
                features={
                    "observation.image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channels"],
                    },
                    "observation.wrist_image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channel"],
                    },
                    "observation.state": {
                        "dtype": "float32",
                        "shape": (6,),
                        "names": ["state"], # x, y, z, roll, pitch, yaw
                    },
                    "action": {
                        "dtype": "float32",
                        "shape": (7,),
                        "names": ["action"], # 6 joint angles and 1 gripper
                    },
                    "obj_init": {
                        "dtype": "float32",
                        "shape": (9,),
                        "names": ["obj_init"], # just the initial position of the object. Not used in training.
                    },
                },
                image_writer_threads=10,
                image_writer_processes=5,
        )
else:
    print("Load from previous dataset")
    dataset = LeRobotDataset(REPO_NAME, root=ROOT)

Directory ./demo_data_language already exists.


## Keyboard Control
You can teleop your robot with keyboard and collect dataset
```
---------     -----------------------
   w       ->        backward
s  a  d        left   forward   right
---------      -----------------------
In x, y plane

---------
R: Moving Up
F: Moving Down
---------
In z axis

---------
Q: Tilt left
E: Tilt right
UP: Look Upward
Down: Look Donward
Right: Turn right
Left: Turn left
---------
For rotation

---------
SPACEBAR: Toggle Gripper
--------

---------
z: reset
--------
```
Reseting your environment will remove the cache data of the current demonstration and restart collection.

### Now let's teleop our robot and collect data!

**To receive the success signal, you have to release the gripper and move upwards above the mug!**

In [7]:
action = np.zeros(7)
episode_id = 0
record_flag = False # Start recording when the robot starts moving
while PnPEnv.env.is_viewer_alive() and episode_id < NUM_DEMO:
    PnPEnv.step_env()
    if PnPEnv.env.loop_every(HZ=20):
        # check if the episode is done
        done = PnPEnv.check_success()
        if done: 
            # Save the episode data and reset the environment
            dataset.save_episode()
            PnPEnv.reset()
            episode_id += 1
        # Teleoperate the robot and get delta end-effector pose with gripper
        action, reset  = PnPEnv.teleop_robot()
        if not record_flag and sum(action) != 0:
            record_flag = True
            print("Start recording")
        if reset:
            # Reset the environment and clear the episode buffer
            # This can be done by pressing 'z' key
            # PnPEnv.reset(seed=SEED)
            PnPEnv.reset()
            dataset.clear_episode_buffer()
            record_flag = False
        # Step the environment
        # Get the end-effector pose and images
        agent_image,wrist_image = PnPEnv.grab_image()
        # # resize to 256x256
        agent_image = Image.fromarray(agent_image)
        wrist_image = Image.fromarray(wrist_image)
        agent_image = agent_image.resize((256, 256))
        wrist_image = wrist_image.resize((256, 256))
        agent_image = np.array(agent_image)
        wrist_image = np.array(wrist_image)
        joint_q = PnPEnv.step(action)
        action = PnPEnv.q[:7] # 6 joint angles and 1 gripper
        action = action.astype(np.float32)
        if record_flag:
            # Add the frame to the dataset
            dataset.add_frame( {
                    "observation.image": agent_image,
                    "observation.wrist_image": wrist_image,
                    "observation.state": joint_q[:6], 
                    "action": action,
                    "obj_init": PnPEnv.obj_init_pose,
                    # "task": PnPEnv.instruction,
                }, task = PnPEnv.instruction
            )
        PnPEnv.render(teleop=True, idx=episode_id)

Start recording


Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 95.18ba/s]


随机生成red block位置: [0.324, -0.326, 0.840]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION


Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 79.70ba/s]


随机生成red block位置: [0.340, -0.199, 0.840]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION


...

随机生成red block位置: [0.231, -0.251, 0.840]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION


Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 99.17ba/s]


随机生成red block位置: [0.362, -0.242, 0.840]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION


Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 83.68ba/s]


随机生成red block位置: [0.243, -0.223, 0.840]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION


In [8]:
# ==================== 使用总结 ====================

print("🎯 随机Block位置数据收集 - 使用总结")
print("=" * 60)
print()
print("📋 数据收集流程：")
print("1️⃣ 在Cell 3中配置参数：")
print("   - ENABLE_RANDOM_BLOCK: 启用/禁用随机位置")
print("   - NUM_DEMO: 要收集的episode数量") 
print("   - SEED: 随机种子设置")
print("2️⃣ 运行Cell 3更新配置")
print("3️⃣ 运行Cell 4创建环境")
print("4️⃣ 运行Cell 5测试随机位置生成")
print("5️⃣ 运行Cell 6创建数据集")
print("6️⃣ 运行Cell 12开始数据收集")
print()
print("✨ 新功能特点：")
print("   - 🎯 基于精确几何分析的随机位置生成")
print("   - 🔒 确保block四个角都在plate平整区域内")
print("   - 🎮 保持原有的键盘控制体验")
print("   - 🔄 支持固定位置和随机位置切换")
print("   - 📊 自动显示位置分布统计")
print()
print("🎯 推荐数据收集策略：")
strategies = [
    "策略1: 先收集固定位置基线数据 (ENABLE_RANDOM_BLOCK=False)",
    "策略2: 再收集随机位置数据增强泛化性 (ENABLE_RANDOM_BLOCK=True)",
    "策略3: 使用不同随机种子收集多组数据",
    "策略4: 根据训练效果调整NUM_DEMO数量"
]
for strategy in strategies:
    print(f"   {strategy}")

print()
print("🔧 参数调优建议：")
print("   - 初学者: ENABLE_RANDOM_BLOCK=False, NUM_DEMO=20")
print("   - 进阶用户: ENABLE_RANDOM_BLOCK=True, NUM_DEMO=50")
print("   - 研究用途: 收集多组不同配置的数据集")
print()
print("💡 提示：每次修改配置后，重新运行Cell 3-5即可！")
print("=" * 60)


🎯 随机Block位置数据收集 - 使用总结

📋 数据收集流程：
1️⃣ 在Cell 3中配置参数：
   - ENABLE_RANDOM_BLOCK: 启用/禁用随机位置
   - NUM_DEMO: 要收集的episode数量
   - SEED: 随机种子设置
2️⃣ 运行Cell 3更新配置
3️⃣ 运行Cell 4创建环境
4️⃣ 运行Cell 5测试随机位置生成
5️⃣ 运行Cell 6创建数据集
6️⃣ 运行Cell 12开始数据收集

✨ 新功能特点：
   - 🎯 基于精确几何分析的随机位置生成
   - 🔒 确保block四个角都在plate平整区域内
   - 🎮 保持原有的键盘控制体验
   - 🔄 支持固定位置和随机位置切换
   - 📊 自动显示位置分布统计

🎯 推荐数据收集策略：
   策略1: 先收集固定位置基线数据 (ENABLE_RANDOM_BLOCK=False)
   策略2: 再收集随机位置数据增强泛化性 (ENABLE_RANDOM_BLOCK=True)
   策略3: 使用不同随机种子收集多组数据
   策略4: 根据训练效果调整NUM_DEMO数量

🔧 参数调优建议：
   - 初学者: ENABLE_RANDOM_BLOCK=False, NUM_DEMO=20
   - 进阶用户: ENABLE_RANDOM_BLOCK=True, NUM_DEMO=50
   - 研究用途: 收集多组不同配置的数据集

💡 提示：每次修改配置后，重新运行Cell 3-5即可！


In [9]:
PnPEnv.env.close_viewer()